# Download and export openFIGI data for S&P100 stocks

In [11]:
import pandas as pd
import json
from figi import map_jobs, insert_data_into_db

### Load the S&P100 ticker and related identification data

The composition of the S&P100 index is taken as of March 18, 2024. 

S&P100 ticker, name, and sector data source: [wikipedia](https://en.wikipedia.org/wiki/S%26P_100)

ISINs were collected manually

In [12]:
with open('../../data/sp100_constituents.json', 'r') as file:
    companies = json.load(file)

df_base = pd.DataFrame.from_dict(companies)

In [13]:
df_base.head()

,name,sector,isin,ticker
0,Apple,Information Technology,US0378331005,AAPL
1,AbbVie,Health Care,US00287Y1091,ABBV
2,Abbott Laboratories,Health Care,US0028241000,ABT
3,Accenture,Information Technology,IE00B4BNMY34,ACN
4,Adobe,Information Technology,US00724F1012,ADBE


### Fetch FIGI codes from openFIGI

For details on openFIGI API, see the official [webpage](https://www.openfigi.com/api)

For details on exchange codes and their meanings, see the downlaodable file called Exchange Codes in this [page](https://www.openfigi.com/about/figi). A copy of the file is available in this directory under the name *openfigi_exchange_codes.csv*

### Create a list of openFIGI input objects with ISIN values and New York Stock Exchange MIC code XNYS

In [14]:
mic_code = 'XNYS' # we are interested in the records for the New York Stock Exchange market
open_figi_jobs = [{'idType': 'TICKER', 'micCode': mic_code, 'idValue': company['ticker']} for company in companies]

### Make the API calls

In [15]:
open_figi_results = []
chunk_size = 10  # The free openFIGI API has a rate limit of 10 max jobs per request
for i in range(0, len(open_figi_jobs), chunk_size): # Loop over the list, processing 10 items at a time
    jobs_chunk = open_figi_jobs[i:i+chunk_size]
    open_figi_results.extend(map_jobs(jobs_chunk))

### Clean the data

In [16]:
open_figi_results = [i for i in open_figi_results if 'error' not in i]
open_figi_results = [d['data'][0] for d in open_figi_results]

### Create a dataframe and drop unnecessary columns

In [17]:

df_figi = pd.DataFrame.from_dict(open_figi_results)

df_figi.rename(columns={"exchCode": "exch_code", 
                        "compositeFIGI": "composite_figi",
                        "securityType": "security_type",
                        "marketSector": "market_sector",
                        "shareClassFIGI": "share_class_figi",
                        "securityType2": "security_type2",
                        "securityDescription": "security_description"},
              inplace=True)
df_figi.drop(columns=['uniqueID', 'uniqueIDFutOpt'], inplace=True)

df_figi.head()

,figi,name,ticker,exch_code,composite_figi,security_type,market_sector,share_class_figi,security_type2,security_description
0,BBG000B9XVV8,APPLE INC,AAPL,UN,BBG000B9XRY4,Common Stock,Equity,BBG001S5N8V8,Common Stock,AAPL
1,BBG003P7K0B5,ABBVIE INC,ABBV,UN,BBG0025Y4RY4,Common Stock,Equity,BBG0025Y4RZ3,Common Stock,ABBV
2,BBG000B9ZZG4,ABBOTT LABORATORIES,ABT,UN,BBG000B9ZXB4,Common Stock,Equity,BBG001S5N9M6,Common Stock,ABT
3,BBG000D9DC22,ACCENTURE PLC-CL A,ACN,UN,BBG000D9D830,Common Stock,Equity,BBG001SCXK90,Common Stock,ACN
4,BBG000BB5159,ADOBE INC,ADBE,UN,BBG000BB5006,Common Stock,Equity,BBG001S5NCQ5,Common Stock,ADBE


### Add the MIC code column

In [18]:
df_figi['mic_code'] = mic_code
df_figi = df_figi.merge(df_base[['ticker', 'isin']], on='ticker', how='left')

In [19]:
df_figi.head()

,figi,name,ticker,exch_code,composite_figi,security_type,market_sector,share_class_figi,security_type2,security_description,mic_code,isin
0,BBG000B9XVV8,APPLE INC,AAPL,UN,BBG000B9XRY4,Common Stock,Equity,BBG001S5N8V8,Common Stock,AAPL,XNYS,US0378331005
1,BBG003P7K0B5,ABBVIE INC,ABBV,UN,BBG0025Y4RY4,Common Stock,Equity,BBG0025Y4RZ3,Common Stock,ABBV,XNYS,US00287Y1091
2,BBG000B9ZZG4,ABBOTT LABORATORIES,ABT,UN,BBG000B9ZXB4,Common Stock,Equity,BBG001S5N9M6,Common Stock,ABT,XNYS,US0028241000
3,BBG000D9DC22,ACCENTURE PLC-CL A,ACN,UN,BBG000D9D830,Common Stock,Equity,BBG001SCXK90,Common Stock,ACN,XNYS,IE00B4BNMY34
4,BBG000BB5159,ADOBE INC,ADBE,UN,BBG000BB5006,Common Stock,Equity,BBG001S5NCQ5,Common Stock,ADBE,XNYS,US00724F1012


### Insert data into the database

In [20]:
insert_data_into_db(df_figi)

Successfully inserted 99 rows into the 'figi' table.
